In [ ]:
import pandas as pd
import numpy as np
import string
import re
import nltk


In [ ]:
### Reading in the cases from crown notes
crown_cases = pd.read_csv('RITM0414461_cases_crownnotes_11172022.csv', sep = '|')


In [ ]:
crown_cases

In [ ]:
# making sure all mrn are intergers
crown_cases['mrn'] = crown_cases['mrn'].astype(int)

In [ ]:
crown_dem = pd.read_csv('RITM0414461_cases_demog_11172022.csv', sep = '|')

In [ ]:
sample_dem = crown_dem[crown_dem['mrn'] == 1004682204]

In [ ]:
sample_dem

In [ ]:
### it seems that the format of the epic notes has errors so we need a different way to load it. 
### Loop the data lines
with open('RITM0414461_cases_epicnotes_11172022.csv', 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split("|")) for l in temp_f.readlines() ]

### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
column_names = [i for i in range(0, max(col_count))]

### Read csv
epic_cases = pd.read_csv('RITM0414461_cases_epicnotes_11172022.csv', header=None, delimiter="|", names=column_names)


In [ ]:
epic_cases

In [ ]:
epic_cases[epic_cases[6].notna()]

In [ ]:
### Reformatting to match our desired output
epic = epic_cases[[0,1,2,3,4]]
new_header = epic.iloc[0] # grab first row as column titles
new_header[1] = 'primary_time' # change this to match format of crown notes
epic_cases = epic[1:] # take all data except for our headere row
epic_cases.columns = new_header
epic_cases

In [ ]:
### Joining the data from crown and epic together
combined_cases = pd.concat([crown_cases, epic_cases], axis = 0)

In [ ]:
combined_cases['mrn_int'] = combined_cases['mrn'].astype(int)

In [ ]:
combined_cases

In [ ]:
### This is the number of total cases wee have
combined_cases['mrn_int'].nunique()

In [ ]:
### Filtering out any note that contains .TIF files
combined_cases = combined_cases[~combined_cases['note_text'].str.contains('.TIF')]

In [ ]:
### Taking a look at all other note types
combined_cases['note_type'].unique()

In [ ]:
### The following are examples of the not types we will be including in our analysis

In [ ]:
notes = ['CROWN DOCUMENT: FOLLOW-UP VISIT', 'CROWN DOCUMENT: CONSULT VISIT', 'CROWN DOCUMENT: LETTER(37)', 
        'CROWN DOCUMENT: INITIAL VISIT(237)', 'CROWN DOCUMENT: DERM ONCOLOGY FOLLOW-UP VISIT', 
         'CROWN DOCUMENT: SKIN CHECK', 'CROWN DOCUMENT: OFFICE VISIT (4992)', 'CROWN DOCUMENT: DERM ONCOLOGY INITIAL VISIT',
        'Progress Note']

In [ ]:
### Filter our combined dataset to only include notes in our notes variable
combined_cases = combined_cases[combined_cases['note_type'].isin(notes)]

In [ ]:
combined_cases['mrn_int'].nunique()

In [ ]:
combined_cases['target'] = 'ctcl'

In [ ]:
combined_cases

In [ ]:
combined_cases.to_csv('cases_filtered.csv', index = False)